In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.pandas.set_option('display.max_columns',None)

In [10]:
df = pd.read_csv('Feature_Engg.csv')

In [11]:
df.head()

,education,balance,day,month,pdays,previous,y,unemployed,unknown_employment,single,Default,Housing,Loan,Called,days_since_contacted,months_last_contacted,box_cox_duration,Contact_campaign,Previous_outcome_1,Previous_outcome_2,Age
0,3,1,5,5,0,0,0,0,0,0,0,1,0,1,26,7,3.043183,1,0,1,0
1,2,0,5,5,0,0,0,0,0,1,0,1,0,1,26,7,2.727692,1,0,1,0
2,2,0,5,5,0,0,0,0,0,0,0,1,1,1,26,7,2.377731,1,0,1,1
3,3,0,5,5,0,0,0,0,0,0,0,1,0,1,26,7,2.682891,1,0,1,1
4,3,0,5,5,0,0,0,0,0,1,0,1,1,1,26,7,2.932864,1,0,1,1


In [12]:
X = df.drop('y',axis=1).values
y=df['y'].values

In [13]:
X.shape

(43354, 20)

In [14]:
y.shape

(43354,)

In [15]:
##Train Test Split

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
##Random over sampling

In [19]:
from imblearn.over_sampling import RandomOverSampler

In [21]:
ros = RandomOverSampler(random_state=0)

In [23]:
X_train_res,y_train_res = ros.fit_resample(X_train,y_train)

In [24]:
X_train_res.shape

(51428, 20)

In [25]:
y_train_res.shape

(51428,)

# RandomForest

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [28]:
rfc = RandomForestClassifier()

In [36]:
params_rfc = {'n_estimators':[100,500,700],
             'max_depth':[5,7,20,30]}

In [37]:
RSCV_rfc = RandomizedSearchCV(rfc,params_rfc,cv=5,verbose=1)

In [38]:
fitted_rfc_rscv= RSCV_rfc.fit(X_train_res,y_train_res)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  8.4min finished


In [39]:
fitted_rfc_rscv.best_score_

0.9653301611821213

In [40]:
fitted_rfc_rscv.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [41]:
rfc_tuned = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
rfc_tuned.fit(X_train_res,y_train_res)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
predictions_rfc = rfc_tuned.predict(X_test)

In [52]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,roc_curve

In [45]:
print(classification_report(y_test,predictions_rfc))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94     12603
           1       0.59      0.56      0.58      1704

    accuracy                           0.90     14307
   macro avg       0.77      0.76      0.76     14307
weighted avg       0.90      0.90      0.90     14307



In [46]:
print(confusion_matrix(y_test,predictions_rfc))

[[11935   668]
 [  742   962]]


In [47]:
print(roc_auc_score(predictions_rfc,y_test))

0.7658264254233672


In [48]:
from sklearn.metrics import accuracy_score

In [49]:
accuracy_score(y_test,predictions_rfc)

0.9014468442021388

In [50]:
## 90% accuracy

# XGBoost

In [54]:
from xgboost import XGBClassifier

In [55]:
xgb_clf = XGBClassifier()

In [56]:
params = {'max_depth':[3,7,10,12],
         'learning_rate':[0.1,0.001,0.01],
         'n_estimators':[100,300,500,1000]}

In [72]:
rsc_xgb = RandomizedSearchCV(xgb_clf,params,cv=5,verbose=1,scoring='precision')

In [73]:
fiited_rsc_xgb = rsc_xgb.fit(X_train_res,y_train_res)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 13.2min finished


In [76]:
fiited_rsc_xgb.best_score_

0.9282616271344521

In [75]:
fiited_rsc_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [77]:
tuned_xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [78]:
tuned_xgb.fit(X_train_res,y_train_res)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [79]:
xgb_predictions = tuned_xgb.predict(X_test)

In [80]:
confusion_matrix(xgb_predictions,y_test)

array([[11783,   660],
       [  820,  1044]], dtype=int64)

In [81]:
print(classification_report(xgb_predictions,y_test))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94     12443
           1       0.61      0.56      0.59      1864

    accuracy                           0.90     14307
   macro avg       0.77      0.75      0.76     14307
weighted avg       0.89      0.90      0.89     14307



In [82]:
accuracy_score(xgb_predictions,y_test)

0.8965541343398337

In [83]:
roc_auc_score(xgb_predictions,y_test)

0.7535219829892118